In [536]:
import pickle
# import joblib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from lightgbm import LGBMRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [537]:
df = pd.read_excel('/kaggle/input/rzd-train-dataset/dataset_train.xlsx')
df

,Наименование полигона,Краткое наименование,Полигон,Номерной знак ТС,Наименование структурного подразделения,Тип закрепления,дата путевого листа,"Данные путевых листов, пробег",Дата сигнала телематики,"Данные телематики, пробег",Штрафы,манера вождения,путевые листы,соответствие целевой структуре,Рейтинг
0,ОКТЯБРЬСКАЯ ЖД,ЦМ,Окт ДМ,0600РХ70,ГТС Выборг,В целевой структуре парка,2024-04-01,116.0,2024-04-01,116.08,7,2.6,1.00,1.39,1.39
1,ОКТЯБРЬСКАЯ ЖД,ЦМ,Окт ДМ,0010ТМ69,Московская механизированная дистанция ОКТ ДМ (...,В целевой структуре парка,2024-04-01,0.0,2024-04-01,0.00,0,0.0,1.00,1.24,0.62
2,ОКТЯБРЬСКАЯ ЖД,ЦМ,Окт ДМ,0003ТВ69,Московская механизированная дистанция ОКТ ДМ (...,В целевой структуре парка,2024-04-01,90.2,2024-04-01,140.12,13,5.7,0.64,1.24,1.74
3,ОКТЯБРЬСКАЯ ЖД,ЦМ,Окт ДМ,М015НН197,Московская механизированная дистанция ОКТ ДМ (...,В целевой структуре парка,2024-04-01,600.0,2024-04-01,589.99,3,5.6,1.02,1.24,0.78
4,ОКТЯБРЬСКАЯ ЖД,ЦМ,Окт ДМ,4947ТВ69,ГТС Выборг,Прочие,2024-04-01,0.0,2024-04-01,0.00,0,0.0,1.02,1.39,0.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,СЕВЕРНАЯ ЖД,ЦМ,Северная ДМ,Р065ХН07,Архангельская механизированная дистанция ПРР и КО,В целевой структуре парка,NaT,12367.0,NaT,1246.78,1,3.6,9.92,1.47,2.75
439,СЕВЕРНАЯ ЖД,ЦМ,Северная ДМ,С741ХУ07,Архангельская механизированная дистанция ПРР и КО,В целевой структуре парка,NaT,11910.0,NaT,11917.10,6,4.8,1.00,1.47,1.29
440,СЕВЕРНАЯ ЖД,ЦМ,Северная ДМ,К113ВТ07,Архангельская механизированная дистанция ПРР и КО,В целевой структуре парка,NaT,17015.0,NaT,17019.13,0,6.0,1.00,1.47,1.47
441,СЕВЕРНАЯ ЖД,ЦМ,Северная ДМ,Р390ХН07,Архангельская механизированная дистанция ПРР и КО,В целевой структуре парка,NaT,2281.0,NaT,2310.52,0,5.2,0.99,1.47,0.50


In [538]:
df_class = df[['Номерной знак ТС', 'Наименование структурного подразделения', 'Данные путевых листов, пробег', 'Данные телематики, пробег', 'Тип закрепления']].copy()
df_class

,Номерной знак ТС,Наименование структурного подразделения,"Данные путевых листов, пробег","Данные телематики, пробег",Тип закрепления
0,0600РХ70,ГТС Выборг,116.0,116.08,В целевой структуре парка
1,0010ТМ69,Московская механизированная дистанция ОКТ ДМ (...,0.0,0.00,В целевой структуре парка
2,0003ТВ69,Московская механизированная дистанция ОКТ ДМ (...,90.2,140.12,В целевой структуре парка
3,М015НН197,Московская механизированная дистанция ОКТ ДМ (...,600.0,589.99,В целевой структуре парка
4,4947ТВ69,ГТС Выборг,0.0,0.00,Прочие
...,...,...,...,...,...
438,Р065ХН07,Архангельская механизированная дистанция ПРР и КО,12367.0,1246.78,В целевой структуре парка
439,С741ХУ07,Архангельская механизированная дистанция ПРР и КО,11910.0,11917.10,В целевой структуре парка
440,К113ВТ07,Архангельская механизированная дистанция ПРР и КО,17015.0,17019.13,В целевой структуре парка
441,Р390ХН07,Архангельская механизированная дистанция ПРР и КО,2281.0,2310.52,В целевой структуре парка


In [540]:
df_class.replace('прочие', 'Прочие', inplace=True)

['В целевой структуре парка' 'Прочие' 'Проверить данное ТС']


In [542]:
df_class['Тип закрепления'] = df_class['Тип закрепления'].astype(str)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443 entries, 0 to 442
Data columns (total 5 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Номерной знак ТС                         443 non-null    object 
 1   Наименование структурного подразделения  443 non-null    object 
 2   Данные путевых листов, пробег            443 non-null    float64
 3   Данные телематики, пробег                443 non-null    float64
 4   Тип закрепления                          443 non-null    object 
dtypes: float64(2), object(3)
memory usage: 17.4+ KB


In [543]:
label_encoder = LabelEncoder()
df_class.loc[:, 'Наименование структурного подразделения'] = label_encoder.fit_transform(df_class['Наименование структурного подразделения'])

In [544]:
label_encoder = LabelEncoder()
df_class.loc[:, 'Номерной знак ТС'] = label_encoder.fit_transform(df_class['Номерной знак ТС'])

In [545]:
label_encoder = LabelEncoder()
df_class.loc[:, 'Тип закрепления'] = label_encoder.fit_transform(df_class['Тип закрепления'])

In [547]:
Q1 = df_class['Данные путевых листов, пробег'].quantile(0.25)
Q3 = df_class['Данные путевых листов, пробег'].quantile(0.75)
IQR = Q3 - Q1

df_class = df_class[~((df_class['Данные путевых листов, пробег'] < (Q1 - 1.5 * IQR)) |(df_class['Данные путевых листов, пробег'] > (Q3 + 1.5 * IQR)))]

<class 'pandas.core.frame.DataFrame'>
Index: 361 entries, 0 to 442
Data columns (total 5 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Номерной знак ТС                         361 non-null    object 
 1   Наименование структурного подразделения  361 non-null    object 
 2   Данные путевых листов, пробег            361 non-null    float64
 3   Данные телематики, пробег                361 non-null    float64
 4   Тип закрепления                          361 non-null    object 
dtypes: float64(2), object(3)
memory usage: 16.9+ KB


In [548]:
Q1 = df_class['Данные телематики, пробег'].quantile(0.25)
Q3 = df_class['Данные телематики, пробег'].quantile(0.75)
IQR = Q3 - Q1

df_class = df_class[~((df_class['Данные телематики, пробег'] < (Q1 - 1.5 * IQR)) |(df_class['Данные телематики, пробег'] > (Q3 + 1.5 * IQR)))]

<class 'pandas.core.frame.DataFrame'>
Index: 325 entries, 0 to 442
Data columns (total 5 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Номерной знак ТС                         325 non-null    object 
 1   Наименование структурного подразделения  325 non-null    object 
 2   Данные путевых листов, пробег            325 non-null    float64
 3   Данные телематики, пробег                325 non-null    float64
 4   Тип закрепления                          325 non-null    object 
dtypes: float64(2), object(3)
memory usage: 15.2+ KB


In [550]:
df_class['Тип закрепления'] = df_class['Тип закрепления'].astype(str)

In [552]:
train = df_class
train = train.drop(test.index)

y = train['Тип закрепления']
train = train.drop('Тип закрепления', axis=1)
X = train

,Номерной знак ТС,Наименование структурного подразделения,"Данные путевых листов, пробег","Данные телематики, пробег"
0,63,2,116.0,116.08
1,6,9,0.0,0.00
2,3,9,90.2,140.12
3,337,9,600.0,589.99
4,163,2,0.0,0.00
...,...,...,...,...
417,21,0,0.0,23.20
423,1,0,0.0,204.36
424,300,0,0.0,0.00
432,333,0,0.0,0.00


In [509]:
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

In [555]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [556]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [485]:
base_classifier = KNeighborsClassifier(n_neighbors=3)
bagging_clf = BaggingClassifier(estimator=base_classifier, n_estimators=10, random_state=42)
bagging_clf.fit(X_train_scaled, y_train)
predicrions_class = bagging_clf.predict(X_test)

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but BaggingClassifier was fitted without feature names
  warnings.warn(


In [486]:
accuracy = accuracy_score(y_test, predicrions_class)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.43


In [557]:
random_forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_clf.fit(X_train_scaled, y_train)
predicrions_class = random_forest_clf.predict(X_test_scaled)

In [558]:
accuracy = accuracy_score(y_test, predicrions_class)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.78


In [559]:
test_x = test
test_x = test_x.drop('Тип закрепления', axis=1)
test_y = test['Тип закрепления']

,Номерной знак ТС,Наименование структурного подразделения,"Данные путевых листов, пробег","Данные телематики, пробег"
284,161,3,0.0,15.06
123,4,10,461.0,525.62
302,93,5,200.0,336.00
9,30,2,163.0,102.36
105,260,10,157.0,0.00
263,121,3,0.0,581.00
370,171,7,47.0,26.62
242,111,5,238.0,0.00
247,9,3,0.0,0.00
114,257,1,244.0,8.99


In [561]:
test_x_scaled = scaler.fit_transform(test_x)

In [517]:
test_predictions_class = bagging_clf.predict(test_x_scaled)
accuracy = accuracy_score(test_y, test_predictions_class)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.57


In [562]:
test_predictions_class = random_forest_clf.predict(test_x_scaled)
accuracy = accuracy_score(test_y, test_predictions_class)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.73


In [563]:
filename = 'rzd_trgt_strctr_prediction_random_forest_model_1.pkl'
pickle.dump(random_forest_clf, open(filename, 'wb'))

In [87]:
df = df[['Наименование структурного подразделения', 'путевые листы', 'соответствие целевой структуре', 'Штрафы', 'манера вождения', 'Рейтинг']]

,Наименование структурного подразделения,путевые листы,соответствие целевой структуре,Штрафы,манера вождения,Рейтинг
0,ГТС Выборг,1.00,1.39,7,2.6,1.39
1,Московская механизированная дистанция ОКТ ДМ (...,1.00,1.24,0,0.0,0.62
2,Московская механизированная дистанция ОКТ ДМ (...,0.64,1.24,13,5.7,1.74
3,Московская механизированная дистанция ОКТ ДМ (...,1.02,1.24,3,5.6,0.78
4,ГТС Выборг,1.02,1.39,0,0.0,0.66
...,...,...,...,...,...,...
438,Архангельская механизированная дистанция ПРР и КО,9.92,1.47,1,3.6,2.75
439,Архангельская механизированная дистанция ПРР и КО,1.00,1.47,6,4.8,1.29
440,Архангельская механизированная дистанция ПРР и КО,1.00,1.47,0,6.0,1.47
441,Архангельская механизированная дистанция ПРР и КО,0.99,1.47,0,5.2,0.50


In [97]:
label_encoder = LabelEncoder()
df.loc[:, 'Наименование структурного подразделения'] = label_encoder.fit_transform(df['Наименование структурного подразделения'])

In [100]:
Q1 = df['путевые листы'].quantile(0.25)
Q3 = df['путевые листы'].quantile(0.75)
IQR = Q3 - Q1

df = df[~((df['путевые листы'] < (Q1 - 1.5 * IQR)) |(df['путевые листы'] > (Q3 + 1.5 * IQR)))]

<class 'pandas.core.frame.DataFrame'>
Index: 426 entries, 0 to 442
Data columns (total 6 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Наименование структурного подразделения  426 non-null    int64  
 1   путевые листы                            426 non-null    float64
 2   соответствие целевой структуре           426 non-null    float64
 3   Штрафы                                   426 non-null    int64  
 4   манера вождения                          426 non-null    float64
 5   Рейтинг                                  426 non-null    float64
dtypes: float64(4), int64(2)
memory usage: 23.3 KB


In [102]:
Q1 = df['Рейтинг'].quantile(0.25)
Q3 = df['Рейтинг'].quantile(0.75)
IQR = Q3 - Q1

df = df[~((df['Рейтинг'] < (Q1 - 1.5 * IQR)) |(df['Рейтинг'] > (Q3 + 1.5 * IQR)))]

<class 'pandas.core.frame.DataFrame'>
Index: 418 entries, 0 to 442
Data columns (total 6 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Наименование структурного подразделения  418 non-null    int64  
 1   путевые листы                            418 non-null    float64
 2   соответствие целевой структуре           418 non-null    float64
 3   Штрафы                                   418 non-null    int64  
 4   манера вождения                          418 non-null    float64
 5   Рейтинг                                  418 non-null    float64
dtypes: float64(4), int64(2)
memory usage: 22.9 KB


In [104]:
Q1 = df['Штрафы'].quantile(0.25)
Q3 = df['Штрафы'].quantile(0.75)
IQR = Q3 - Q1

df = df[~((df['Штрафы'] < (Q1 - 1.5 * IQR)) |(df['Штрафы'] > (Q3 + 1.5 * IQR)))]

<class 'pandas.core.frame.DataFrame'>
Index: 413 entries, 0 to 442
Data columns (total 6 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   Наименование структурного подразделения  413 non-null    int64  
 1   путевые листы                            413 non-null    float64
 2   соответствие целевой структуре           413 non-null    float64
 3   Штрафы                                   413 non-null    int64  
 4   манера вождения                          413 non-null    float64
 5   Рейтинг                                  413 non-null    float64
dtypes: float64(4), int64(2)
memory usage: 22.6 KB


In [ ]:
test = df.sample(n=30, random_state=42)

In [213]:
train = df
train = train.drop(test.index)

y = train['Рейтинг']
train = train.drop('Рейтинг', axis=1)
X = train
X

,Наименование структурного подразделения,путевые листы,соответствие целевой структуре,Штрафы,манера вождения
0,2,1.00,1.39,7,2.6
1,9,1.00,1.24,0,0.0
2,9,0.64,1.24,13,5.7
3,9,1.02,1.24,3,5.6
4,2,1.02,1.39,0,0.0
...,...,...,...,...,...
435,0,0.74,1.47,1,6.0
439,0,1.00,1.47,6,4.8
440,0,1.00,1.47,0,6.0
441,0,0.99,1.47,0,5.2


In [216]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [217]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [218]:
lgbm_reg = LGBMRegressor(random_state=42)

In [219]:
bagging_regressor = BaggingRegressor(lgbm_reg, n_estimators=10, random_state=42)
bagging_regressor.fit(X_train_scaled, y_train)
predictions = bagging_regressor.predict(X_test_scaled)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000072 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 112
[LightGBM] [Info] Number of data points in the train set: 306, number of used features: 5
[LightGBM] [Info] Start training from score 0.979183
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

In [221]:
test_x = test
test_x = test_x.drop('Рейтинг', axis=1)
test_y = test['Рейтинг']

,Наименование структурного подразделения,путевые листы,соответствие целевой структуре,Штрафы,манера вождения
417,0,0.00,1.47,3,4.8
366,3,0.00,1.95,15,5.9
423,0,0.00,1.47,1,5.8
370,7,1.77,1.35,9,5.9
196,10,0.99,1.52,0,0.0
436,0,0.74,1.47,3,5.6
311,5,0.43,1.46,0,0.0
259,5,0.08,1.46,0,5.9
332,3,1.00,1.95,5,4.9
210,11,0.00,2.25,0,0.0


In [223]:
test_x_scaled = scaler.fit_transform(test_x)

In [228]:
test_predictions = bagging_regressor.predict(test_x_scaled)

In [226]:
r2 = round(r2_score(test_y, test_predictions), 2)
print(f'R2: {r2}')

R2: 0.74


In [227]:
filename = 'rzd_rating_prediction_lgbm_bagging_model.pkl'
pickle.dump(bagging_regressor, open(filename, 'wb'))